In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
rmdir /content/gdrive/MyDrive/Colab Notebooks/CMID_Imbalanced/.ipynb_checkpoints

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.layers as tfl
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
# from resnets_utils import *
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow

import matplotlib.pyplot as plt
import os
import tensorflow.keras.layers as tfl

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation

In [ ]:
BATCH_SIZE = 32
# IMG_SIZE = (160, 160) #for mobileNet v1
IMG_SIZE = (224, 224) #for resnet50
# directory = "dataset/"

directory = "/content/gdrive/MyDrive/Colab Notebooks/CMID/"

train_dataset = image_dataset_from_directory(directory,
                                            #  labels='inferred',
                                             label_mode='binary',
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='training',
                                            #  label_mode = tf.float32,
                                             seed=42)
validation_dataset = image_dataset_from_directory(directory,
                                                  # labels='inferred',
                                                  label_mode='binary',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  image_size=IMG_SIZE,
                                                  validation_split=0.2,
                                                  subset='validation',
                                                  seed=42)

In [ ]:
train_dataset

In [ ]:
class_names = train_dataset.class_names

In [ ]:
# print(labels)

In [ ]:
print(class_names)

In [ ]:
i = 0
for images, labels in train_dataset: # train_dataset.take(1):
  # print(images.shape, labels.shape)
  if(i==0):
    X_train = images
    y_train = labels
  else:
    X_train = tf.concat([X_train, images], 0)
    y_train = tf.concat([y_train, labels], 0)

  i = i + 1
print(X_train.shape, y_train.shape)

In [ ]:
i = 0
for images, labels in validation_dataset: # train_dataset.take(1):
  # print(images.shape, labels.shape)
  if(i==0):
    X_valid = images
    y_valid = labels
  else:
    X_valid = tf.concat([X_valid, images], 0)
    y_valid = tf.concat([y_valid, labels], 0)

  i = i + 1
print(X_valid.shape, y_valid.shape)

In [ ]:
#SVM on CMID datasets
#dataset preparation
import numpy as np
from skimage.transform import resize
flat_X_train = []
flat_y_train = []
for i in range(0, len(X_train)):
  img_array = X_train[i]
  img_resized = resize(img_array, [224, 224, 3])
  flat_X_train.append(img_resized.flatten())

for i in range(0, len(y_train)):
  flat_y_train.append(int(y_train[i]))

flat_X_train = np.array(flat_X_train)
flat_y_train = np.array(flat_y_train)

flat_X_valid = []
flat_y_valid = []
for i in range(0, len(X_valid)):
  img_array = X_valid[i]
  img_resized = resize(img_array, [224, 224, 3])
  flat_X_valid.append(img_resized.flatten())

for i in range(0, len(y_valid)):
  flat_y_valid.append(int(y_valid[i]))

flat_X_valid = np.array(flat_X_valid)
flat_y_valid = np.array(flat_y_valid)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
flat_X_train = scaler.fit_transform(flat_X_train)
flat_X_valid = scaler.fit_transform(flat_X_valid)
#full dataset
X_main = np.concatenate((flat_X_train, flat_X_valid), axis=0) #flat_X_train.append(flat_X_valid)
y_main = np.concatenate((flat_y_train, flat_y_valid), axis=0) #flat_y_train.append()

In [ ]:
X = X_main; y = y_main #train with ALL the datasets
# X = X_main[0:400]; y = y_main[0:400] #train with FIRST half of the datasets
# X = X_main[400:791]; y = y_main[400:791] #train with SECOND half of the datsets
# test_X = X_main[0:100]; test_y = y_main[0:100] #test datasets
print('X and y shape= ', X.shape, y.shape)
# print('test_X and test_y shape= ', test_X.shape, test_y.shape)

In [ ]:
# ML model training
# from sklearn.model_selection import GridSearchCV
# # param_grid={'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf','poly']}
# param_grid={'C':[1],'gamma':[0.0001],'kernel':['rbf']}

from sklearn import svm
svm_clf = svm.SVC(kernel='rbf', C=1, gamma=0.0001) # for cross validation
# for normal classification
# svc_prob=svm.SVC(probability=True)
# for grid search 
# svm_clf=GridSearchCV(svc_prob,param_grid)

# svm_clf.fit(X, y)
# y_pred_test = svm_clf.predict(test_X)

# from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# print('test acc=', accuracy_score(test_y, y_pred_test))
# print('precision=', np.mean(precision_score(test_y, y_pred_test, average=None)))
# print('recall=   ', np.mean(recall_score(test_y, y_pred_test, average=None)))
# print('f1=       ', np.mean(f1_score(test_y, y_pred_test, average=None)))

In [ ]:
# # ML model training using Random Forest, XGBoost
from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
# # param_grid={'n_estimators':[16, 32, 64, 128],'max_leaf_nodes':[8, 16, 32]}
# param_grid={'n_estimators':[32],'max_leaf_nodes':[16]}

rf_clf = RandomForestClassifier(n_estimators=32, max_leaf_nodes=16, n_jobs=-1)
# for normal classification
# rf_normal=RandomForestClassifier(n_jobs=-1)
#for grid search 
# rf_clf=GridSearchCV(rf_normal,param_grid)

rf_clf.fit(X, y)
# y_pred_test = rf_clf.predict(test_X)

# print('test acc=', accuracy_score(test_y, y_pred_test))
# print('precision=', np.mean(precision_score(test_y, y_pred_test, average=None)))
# print('recall=   ', np.mean(recall_score(test_y, y_pred_test, average=None)))
# print('f1=       ', np.mean(f1_score(test_y, y_pred_test, average=None)))


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
y_pred_test = rf_clf.predict(X)

print('test acc=', accuracy_score(y, y_pred_test))
print('precision=', np.mean(precision_score(y, y_pred_test, average=None)))
print('recall=   ', np.mean(recall_score(y, y_pred_test, average=None)))
print('f1=       ', np.mean(f1_score(y, y_pred_test, average=None)))

In [ ]:
def plot_digit(data):
    image = data.reshape(224, 224, 3) #(28, 28)
    plt.imshow(image, cmap = mpl.cm.hot,
               interpolation="nearest")
    plt.axis("off")

%matplotlib inline
import matplotlib as mpl

plot_digit(rf_clf.feature_importances_)

cbar = plt.colorbar(ticks=[rf_clf.feature_importances_.min(), rf_clf.feature_importances_.max()])
cbar.ax.set_yticklabels(['Not important', 'Very important'])

# save_fig("mnist_feature_importance_plot")
plt.show()

In [ ]:
importances = rf_clf.feature_importances_
importances = importances.reshape((384, -1))

plt.figure(figsize=(10, 5))
plt.imshow(importances)
plt.title("Feature Importance Plot for Color Image (Random Forest)")
plt.axis('off')
plt.colorbar(label='Importance')
plt.show()

In [ ]:
# importances = rf_clf.feature_importances_
importances[importances > 0].shape

In [ ]:
# rf_clf.feature_importances_.max()

for name, score in zip(X[0], rf_clf.feature_importances_):
    print(name, score)

In [ ]:
#cross validation

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, cross_validate

scoring = {'accuracy' : make_scorer(accuracy_score),
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score),
           'f1_score' : make_scorer(f1_score)}

results1 = cross_validate(svm_clf, X, y, cv=5, scoring=scoring, n_jobs=-1)
print('svm acc, std, precision, recall, f1_score=', np.mean(results1['test_accuracy']), np.std(results1['test_accuracy']), np.mean(results1['test_precision']), np.mean(results1['test_recall']), np.mean(results1['test_f1_score']))
results = cross_validate(rf_clf, X, y, cv=5, scoring=scoring, n_jobs=-1)
print('rf acc, std, precision, recall, f1_score=', np.mean(results['test_accuracy']), np.std(results['test_accuracy']), np.mean(results['test_precision']), np.mean(results['test_recall']), np.mean(results['test_f1_score']))


